## 필요 라이브러리 호출

In [1]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from scipy.fft import fft

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import f1_score, classification_report

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐

In [2]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## tag name 출력 파라미터 설정
table = 'bci1'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
tag_name = show_column(NAME_URL)

In [4]:
tag_name

['test-s-0',
 'test-s-1',
 'test-s-10',
 'test-s-11',
 'test-s-12',
 'test-s-13',
 'test-s-14',
 'test-s-15',
 'test-s-16',
 'test-s-17',
 'test-s-18',
 'test-s-19',
 'test-s-2',
 'test-s-20',
 'test-s-21',
 'test-s-22',
 'test-s-23',
 'test-s-24',
 'test-s-25',
 'test-s-26',
 'test-s-27',
 'test-s-28',
 'test-s-29',
 'test-s-3',
 'test-s-30',
 'test-s-31',
 'test-s-32',
 'test-s-33',
 'test-s-34',
 'test-s-35',
 'test-s-36',
 'test-s-37',
 'test-s-38',
 'test-s-39',
 'test-s-4',
 'test-s-40',
 'test-s-41',
 'test-s-42',
 'test-s-43',
 'test-s-44',
 'test-s-45',
 'test-s-46',
 'test-s-47',
 'test-s-48',
 'test-s-49',
 'test-s-5',
 'test-s-50',
 'test-s-51',
 'test-s-52',
 'test-s-53',
 'test-s-54',
 'test-s-55',
 'test-s-56',
 'test-s-57',
 'test-s-58',
 'test-s-59',
 'test-s-6',
 'test-s-60',
 'test-s-61',
 'test-s-62',
 'test-s-63',
 'test-s-7',
 'test-s-8',
 'test-s-9',
 'train-s-0',
 'train-s-1',
 'train-s-10',
 'train-s-11',
 'train-s-12',
 'train-s-13',
 'train-s-14',
 'train-s-1

## TAG Name format 변환 

* 위의 과정에서 BCI1 dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환

In [5]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = tag_name[64:]

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_= ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

'train-s-0','train-s-1','train-s-10','train-s-11','train-s-12','train-s-13','train-s-14','train-s-15','train-s-16','train-s-17','train-s-18','train-s-19','train-s-2','train-s-20','train-s-21','train-s-22','train-s-23','train-s-24','train-s-25','train-s-26','train-s-27','train-s-28','train-s-29','train-s-3','train-s-30','train-s-31','train-s-32','train-s-33','train-s-34','train-s-35','train-s-36','train-s-37','train-s-38','train-s-39','train-s-4','train-s-40','train-s-41','train-s-42','train-s-43','train-s-44','train-s-45','train-s-46','train-s-47','train-s-48','train-s-49','train-s-5','train-s-50','train-s-51','train-s-52','train-s-53','train-s-54','train-s-55','train-s-56','train-s-57','train-s-58','train-s-59','train-s-6','train-s-60','train-s-61','train-s-62','train-s-63','train-s-7','train-s-8','train-s-9','train-s-answer'


##  BCI1 Dataset 로드

* Tag Name 들을 사용하여 데이터 로드

In [6]:
# 데이터 로드 파라미터 설정

# tag table 이름 설정
table = 'bci1'
# tag name 설정
name = quote(tags_, safe=":/")
# 시간 포멧 설정 
timeformat = quote('2006-01-02 15:04:05.000000')
# Train , validation , test 데이터 셋 설정
# 학습 데이터 시작 시간 설정
start_time_train = quote('2024-01-01 00:00:00')
# 학습 데이터  끝 시간 설정
end_time_train = quote('2024-01-01 04:37:03')

In [7]:
def data_load_train(table, tag_name, name, start_time, end_time, timeformat):
    
    # 결과를 저장할 리스트
    result_dfs = []
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')

    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # train-s-answer 값 분리
    df_label = df.iloc[:, -1:].dropna()

    # train-s-answer 컬럼 제거
    df = df.iloc[:, :-1]

    for col_name in tag_name[64:-1]:

        # TIME 설정
        df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')

        # 3초 단위로 그룹화하여 데이터 개수 세기
        df_counts = df.groupby(df['TIME'].dt.floor('3S')).size().reset_index(name='count')

        # 데이터 개수가 동일한 그룹만 필터링
        most_common_count = df_counts['count'].mode()[0]
        filtered_df_counts = df_counts[df_counts['count'] == most_common_count]

        # 필터링된 시간값들을 리스트로 변환
        filtered_times = filtered_df_counts['TIME'].tolist()

        # 원본 데이터프레임에서 필터링된 시간값들만 선택
        filtered_data = df[df['TIME'].dt.floor('3S').isin(filtered_times)]

        # TIME을 기준으로 그룹화 (3초 단위로 반올림)
        filtered_data_ = filtered_data.copy()
        filtered_data_['TIME'] = filtered_data_['TIME'].dt.floor('3S')
        grouped = filtered_data_.groupby('TIME')[col_name].apply(list).reset_index()

        # 리스트를 개별 열로 나누기
        result_df = pd.DataFrame(grouped[col_name].tolist())

        result_dfs.append(result_df)  # 결과 추가
        
    # 결과를 저장할 리스트 초기화
    data_list = []
    k = 0

    for k in result_dfs:
        
        # array 형태로 변환
        data = k.values
        data_list.append(data)

    # 리스트를 NumPy 배열로 변환
    data_array = np.array(data_list)

    # 필요한 형태로 reshape
    # (데이터 개수, 64, 3000) 형태로 변환
    reshaped_array = np.transpose(data_array, (1, 0, 2)) 
    
    # train-s-answer 변경
    df_label.loc[df_label['train-s-answer'] == -1.0, 'train-s-answer'] = 0
    df_label['train-s-answer'] = df_label['train-s-answer'].astype(int)    

    return reshaped_array, df_label

In [8]:
# 데이터 로드

# 학습 데이터 로드
train, train_label = data_load_train(table, tag_name, name, start_time_train, end_time_train, timeformat)

# 데이터 분리
train, test, train_label, test_label = train_test_split(train, train_label, test_size=0.2, random_state=77)
test, valid, test_label, valid_label = train_test_split(test, test_label, test_size=0.5, random_state=77)

train_label = train_label.reset_index(drop=True)
valid_label = valid_label.reset_index(drop=True)

In [9]:
train.shape

(222, 64, 3000)

In [10]:
valid.shape

(28, 64, 3000)

In [11]:
test.shape

(28, 64, 3000)

## 데이터 전처리
   * 1 Hanning Window
   * 2 FFT 
   * 3 PCA

## Hanning Window 적용

In [12]:
# hanning window 함수 설정 
def hanning_window(length):
    return 0.5 * (1 - np.cos(2 * np.pi * np.arange(length) / (length - 1)))

In [13]:
# 파라미터 설정
window_length = 3000

train_ = train * hanning_window(3000)
valid_ = valid * hanning_window(3000)
test_ = test * hanning_window(3000)

## FFT 적용

In [14]:
# FFT 변환 함수
def change_fft(sample_rate, data):
    # 신호의 총 샘플 수
    N = sample_rate
    
    # 각 채널에 대해 FFT 결과를 저장할 배열 초기화
    fft_results = np.zeros((data.shape[0], data.shape[1], N // 2 + 1), dtype=float)
    
    # 전체 데이터에 대해 FFT 적용
    for i in range(data.shape[0]):  # 각 샘플에 대해
        for j in range(data.shape[1]):  # 각 채널에 대해
            yf = fft(data[i, j], n=N)  # FFT 계산
            # FFT 결과의 절댓값을 계산하고 정규화 (유의미한 부분만)
            fft_results[i, j] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    return fft_results

In [15]:
# 샘플링 주기 -> 초당 데이터 개수 
sampling_rate = 3000

# FFT 변환
train_ = change_fft(sampling_rate, train_)
valid_ = change_fft(sampling_rate, valid_)
test_ = change_fft(sampling_rate, test_)

## PCA 적용

In [16]:
# PCA 적용 함수 
def apply_pca(train_data, valid_data, test_data, n_components=0.95):
    
    # 2차원으로 변환
    train_reshaped = train_data.reshape(-1, train_data.shape[2])
    valid_reshaped = valid_data.reshape(-1, valid_data.shape[2])
    test_reshaped = test_data.reshape(-1, test_data.shape[2]) 

    # 훈련 데이터로 PCA 학습
    pca = PCA(n_components=n_components)
    train_pca = pca.fit_transform(train_reshaped)
    
    # 검증 및 테스트 데이터에 PCA 적용
    valid_pca = pca.transform(valid_reshaped)
    test_pca = pca.transform(test_reshaped)

    # 다시 원래의 3차원 형태로 변환
    train_pca_reshaped = train_pca.reshape(train_data.shape[0], train_data.shape[1], -1)
    valid_pca_reshaped = valid_pca.reshape(valid_data.shape[0], valid_data.shape[1], -1)
    test_pca_reshaped = test_pca.reshape(test_data.shape[0], test_data.shape[1], -1)
    
    return train_pca_reshaped, valid_pca_reshaped, test_pca_reshaped, pca

# 데이터셋에 PCA 적용
train_pca, valid_pca, test_pca, pca_model = apply_pca(train_, valid_, test_, n_components=0.95)

## 데이터 셋 & 로더 설정

In [17]:
# 데이터 셋 설정
class EEGDataset(Dataset):
    def __init__(self, data, labels):
        
        self.data = torch.Tensor(data)
        self.labels = torch.Tensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_time_data = self.data[index]
        label = self.labels[index]

        return input_time_data, label

In [18]:
# 데이터 셋 설정 
train_ = EEGDataset(train_pca, train_label.values)
valid_ = EEGDataset(valid_pca, valid_label.values)
test_ = EEGDataset(test_pca, test_label.values)

# 데이터 로더 설정
train_dataloader = DataLoader(train_, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_, batch_size=1, shuffle=False)

In [19]:
print(list(train_dataloader)[0][0].shape)

torch.Size([16, 64, 14])


## 학습 모델 설정 

* ResNet 1d 기본 모델 사용

In [20]:
## ResNet 1d 모델 설정 
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        # Identity mapping
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64 

        
        self.conv1 = nn.Conv1d(64, 64, kernel_size=7, stride=2, padding=3) 
        self.bn1 = nn.BatchNorm1d(64)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1])
        self.layer3 = self._make_layer(block, 256, layers[2])
        self.layer4 = self._make_layer(block, 512, layers[3])
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks):
        layers = []
        layers.append(block(self.in_channels, out_channels))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [21]:
# 모델 설정 파라미터
# 학습률 
learning_rate = 0.01

# 모델 초기화
model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=2).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

ResNet(
  (conv1): Conv1d(64, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequ

## 모델 학습 설정

* 학습 중 validation 데이터 기준 F1 score 값이 제일 높은 모델을 저장

In [22]:
train_loss = []
train_acc = []
total_step = len(train_dataloader)
epoch_in = trange(100, desc='training')
best_f1= 0

for epoch in epoch_in:
    model.train()
    running_loss = 0.0
    correct = 0
    total=0

    preds_ = []
    targets_ = []

    # 모델 학습 
    for batch_idx, train_data in enumerate(train_dataloader):

        inputs = train_data[0].to(device).float()
        labels = train_data[1].to(device).long().squeeze()
        
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # label 예측 값 설정 
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==labels).item()
        total += labels.size(0)
        
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}, train acc: {(100 * correct / total):.4f}')
    
    # Epoch 마다 validation을 진행해서 가장 좋은 성능을 보이는 모델을 저장 
    with torch.no_grad():
        model.eval()
        
        for batch_idx, valid_data in enumerate(valid_dataloader):

            inputs_v = valid_data[0].to(device).float()
            labels_v = valid_data[1].to(device).long().squeeze() 
            
            outputs_v = model(inputs_v)
            
            # label 예측 값 설정 
            _,pred_v = torch.max(outputs_v, dim=1)
            target_v = labels_v.view_as(pred_v)
            
            preds_.append(pred_v)
            targets_.append(target_v)
            
        # 모든 배치에서 수집된 예측과 라벨을 합침
        preds_ = torch.cat(preds_).detach().cpu().numpy()
        targets_ = torch.cat(targets_).detach().cpu().numpy()
        
        f1score = f1_score(targets_, preds_,  average='macro')
        if best_f1 < f1score:
            best_f1 = f1score
            # 베스트 모델 저장 
            with open("./result/BCI1_ResNet1d_General.txt", "a") as text_file:
                print('epoch=====',epoch, file=text_file)
                print(classification_report(targets_, preds_, digits=4), file=text_file)
            torch.save(model, f'./result/BCI1_ResNet1d_General.pt') 
        epoch_in.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_f1}")

training:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



train loss: 1.4139056674071722, train acc: 49.0991

train loss: 1.1966735869646072, train acc: 51.3514

train loss: 1.1239286561806996, train acc: 50.0000

train loss: 1.0237462978277887, train acc: 54.0541

train loss: 0.9544157181467329, train acc: 55.4054

train loss: 0.9004832179773422, train acc: 66.2162

train loss: 0.8366896386961548, train acc: 83.3333

train loss: 0.7724695115217141, train acc: 88.2883

train loss: 0.7162908397851483, train acc: 88.2883

train loss: 0.6693504722722408, train acc: 90.5405

train loss: 0.6212176911460309, train acc: 94.1441

train loss: 0.5784983822959475, train acc: 96.3964

train loss: 0.547223576266513, train acc: 95.0450

train loss: 0.5140178799284717, train acc: 97.2973

train loss: 0.4824921410557137, train acc: 97.7477

train loss: 0.47120989181517614, train acc: 94.1441

train loss: 0.4532639800597563, train acc: 95.0450

train loss: 0.43208300089013174, train acc: 97.2973

train loss: 0.411094871548436, train acc: 98.6486

train loss:

## 모델 테스트

In [23]:
# 베스트 모델 로드
model_ = torch.load(f'./result/BCI1_ResNet1d_General.pt').to(device)

In [24]:
# 모델 테스트 
preds_test = []
target_test = []
with torch.no_grad():
    model_.eval()
    for batch_idx, test_data in enumerate(test_dataloader):
        inputs_t = test_data[0].to(device).float()
        labels_t = test_data[1].to(device).long().squeeze()
        
        outputs_t = model_(inputs_t)
        
        _,pred_t = torch.max(outputs_t, dim=1)

        preds_test.append(pred_t)
        target_test.append(labels_t.item())
        
    # 모든 배치에서 수집된 예측과 라벨을 합침
    preds_test = torch.cat(preds_test).detach().cpu().numpy()

# 테스트 결과 데이터 프레임 생성 
final = pd.DataFrame(target_test, columns=['label'])
final['pred'] = preds_test

## 모델 성능 확인

In [25]:
print(classification_report(final['label'].values, final['pred'].values))

              precision    recall  f1-score   support

           0       0.82      0.88      0.85        16
           1       0.82      0.75      0.78        12

    accuracy                           0.82        28
   macro avg       0.82      0.81      0.82        28
weighted avg       0.82      0.82      0.82        28

